# Homework0
# Школа GataScientist, декабрь 2022, Андреева Ольга

Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф".
Cохранить ее в pandas dataframe

In [ ]:
import requests
import pandas as pd
import json

In [251]:
#функция для получения общего количества id-шников
def getTotalCount():
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=1&limit=1&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    return  res.json().get('data').get('total')

#функция для чтения пачки id-шников
def readIDs(p_offset, p_limit):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={p_offset}&limit={p_limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    try:
        objects_data = res.json()
        objects_list = objects_data.get('data').get('list')
        return [x.get('objId') for x in objects_list]
    except IndexError:
        return([]);
    
#функция для чтения объекта по id-шнику
def readObjectData (p_ObjID):
    # Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{p_ObjID}'
    res = requests.get(url)
    return res.json().get('data')    

In [253]:
v_offset = 1
c_limit = 100
c_max_limit=500  #getTotalCount() - если надо скачать ВСЕ записи
lst_Objects=[]
df_Objects=pd.DataFrame([])
while True:
    objids = readIDs(v_offset, c_limit) #cчитали порцию id-шников
    
    if len(df_Objects.index)+len(objids)>=c_max_limit:
        print("Алярма! ",c_max_limit, "записей! Мне больше нельзя!")
        objids=objids[:c_max_limit-len(df_Objects.index)]
   
    #читаем каждый объект по ID
    lst_Objects=[]
    for i in (objids):
            v_Object=readObjectData(i) 
            lst_Objects.append(v_Object)
    
    #добавляем пачку объектов в датафрейм
    df_Objects=df_Objects.append(pd.DataFrame(lst_Objects),ignore_index=True)     
    if len(objids)<c_limit or len(df_Objects.index)>=c_max_limit: break
    v_offset += c_limit
   
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Алярма!  500 записей! Мне больше нельзя!


In [254]:
df_Objects.head(5)

,address,airQualityIndexValue,complexShortNm,conclusion,developer,floorMax,floorMin,generalContractorNm,greenAreaIndexValue,hobjId,...,quartography,region,residentialBuildings,rpdIssueDttm,rpdNum,rpdPdfLink,soldOutPerc,transportDistIndex,transportDistIndexValue,wallMaterialShortDesc
0,г Тюмень,NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",20,4,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,39024,...,"{'objLivElem1KCnt': 13, 'objLivElem2KCnt': 21,...",72,1,09-12-2022 15:05,72-000095,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.406985,NaN,NaN,Другое
1,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",10,7,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,37165,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 11, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1.000000,NaN,NaN,Другое
2,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",12,6,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,37201,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 24, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.999984,NaN,NaN,Другое
3,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",1,1,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,34754,...,NaN,72,0,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.917243,NaN,NaN,Другое
4,"г Ставрополь, ул Шпаковская, Доваторцев, д. 10...",NaN,NaN,False,"{'devId': 10296, 'devShortCleanNm': 'CЗ СТРОЙР...",10,2,ООО Стройресурс (ИНН: 2634053119),NaN,117248,...,"{'objLivElem1KCnt': 16, 'objLivElem2KCnt': 37,...",26,1,02-12-2022 12:57,26-000567,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.609216,NaN,NaN,Монолит-кирпич


Cохранить pandas dataframe в excel

In [240]:
df_Objects.to_excel("./df_Objects.xlsx")


In [241]:
#чекчек
df_Objects1=pd.read_excel("./df_Objects.xlsx")
df_Objects1.head(5)

,Unnamed: 0,address,airQualityIndexValue,complexShortNm,conclusion,developer,floorMax,floorMin,generalContractorNm,greenAreaIndexValue,...,quartography,region,residentialBuildings,rpdIssueDttm,rpdNum,rpdPdfLink,soldOutPerc,transportDistIndex,transportDistIndexValue,wallMaterialShortDesc
0,0,г Тюмень,NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",20,4,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,...,"{'objLivElem1KCnt': 13, 'objLivElem2KCnt': 21,...",72,1,09-12-2022 15:05,72-000095,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.406985,NaN,NaN,Другое
1,1,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",10,7,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 11, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1.000000,NaN,NaN,Другое
2,2,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",12,6,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 24, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.999984,NaN,NaN,Другое
3,3,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",1,1,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,...,NaN,72,0,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.917243,NaN,NaN,Другое
4,4,"г Ставрополь, ул Шпаковская, Доваторцев, д. 10...",NaN,NaN,False,"{'devId': 10296, 'devShortCleanNm': 'CЗ СТРОЙР...",10,2,ООО Стройресурс (ИНН: 2634053119),NaN,...,"{'objLivElem1KCnt': 16, 'objLivElem2KCnt': 37,...",26,1,02-12-2022 12:57,26-000567,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.609216,NaN,NaN,Монолит-кирпич


Cохранить pandas dataframe в pickle

In [242]:
df_Objects.to_pickle("./df_Objects.pkl")

In [243]:
#чекчек
df_Objects2=pd.read_pickle("./df_Objects.pkl")
df_Objects2.head(5)

,address,airQualityIndexValue,complexShortNm,conclusion,developer,floorMax,floorMin,generalContractorNm,greenAreaIndexValue,hobjId,...,quartography,region,residentialBuildings,rpdIssueDttm,rpdNum,rpdPdfLink,soldOutPerc,transportDistIndex,transportDistIndexValue,wallMaterialShortDesc
0,г Тюмень,NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",20,4,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,39024,...,"{'objLivElem1KCnt': 13, 'objLivElem2KCnt': 21,...",72,1,09-12-2022 15:05,72-000095,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.406985,NaN,NaN,Другое
1,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",10,7,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,37165,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 11, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1.000000,NaN,NaN,Другое
2,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",12,6,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,37201,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 24, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.999984,NaN,NaN,Другое
3,"г Тюмень, ул Профсоюзная, д. 56",NaN,NaN,False,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",1,1,ООО Транспромжилстрой-2005 (ИНН: 7204097730),NaN,34754,...,NaN,72,0,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.917243,NaN,NaN,Другое
4,"г Ставрополь, ул Шпаковская, Доваторцев, д. 10...",NaN,NaN,False,"{'devId': 10296, 'devShortCleanNm': 'CЗ СТРОЙР...",10,2,ООО Стройресурс (ИНН: 2634053119),NaN,117248,...,"{'objLivElem1KCnt': 16, 'objLivElem2KCnt': 37,...",26,1,02-12-2022 12:57,26-000567,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.609216,NaN,NaN,Монолит-кирпич


Cохранить pandas dataframe в БД

In [244]:
#на некоторые колонки запись в SQL рeгается на тип данных
#поэтому перевожу их в сторки
df_Objects['developer']=df_Objects['developer'].astype('str')
df_Objects['photoRenderDTO']=df_Objects['photoRenderDTO'].astype('str')
df_Objects['quartography']=df_Objects['quartography'].astype('str')
df_Objects['objectTransportInfo']=df_Objects['objectTransportInfo'].astype('str')
df_Objects['metro']=df_Objects['metro'].astype('str')

In [245]:
import sqlite3
conn = sqlite3.connect('df_Objects_DB')
#df_some=pd.DataFrame([[1,2],[3,4]])
#df_Objects[['photoRenderDTO','quartography']].to_sql('df_Objects', conn, if_exists='replace', index = False)
df_Objects.to_sql('df_Objects', conn, if_exists='replace', index = False)
conn.commit()
conn.close()

In [246]:
#чекчек
conn = sqlite3.connect('df_Objects_DB')
df_Objects3=pd.read_sql('select * from df_Objects limit 10', conn)
conn.close()
df_Objects3.head(5)


,address,airQualityIndexValue,complexShortNm,conclusion,developer,floorMax,floorMin,generalContractorNm,greenAreaIndexValue,hobjId,...,quartography,region,residentialBuildings,rpdIssueDttm,rpdNum,rpdPdfLink,soldOutPerc,transportDistIndex,transportDistIndexValue,wallMaterialShortDesc
0,г Тюмень,None,None,0,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",20,4,ООО Транспромжилстрой-2005 (ИНН: 7204097730),None,39024,...,"{'objLivElem1KCnt': 13, 'objLivElem2KCnt': 21,...",72,1,09-12-2022 15:05,72-000095,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.406985,None,None,Другое
1,"г Тюмень, ул Профсоюзная, д. 56",None,None,0,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",10,7,ООО Транспромжилстрой-2005 (ИНН: 7204097730),None,37165,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 11, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1.000000,None,None,Другое
2,"г Тюмень, ул Профсоюзная, д. 56",None,None,0,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",12,6,ООО Транспромжилстрой-2005 (ИНН: 7204097730),None,37201,...,"{'objLivElem1KCnt': 0, 'objLivElem2KCnt': 24, ...",72,1,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.999984,None,None,Другое
3,"г Тюмень, ул Профсоюзная, д. 56",None,None,0,"{'devId': 306, 'devShortCleanNm': '2МЕН ГРУПП'...",1,1,ООО Транспромжилстрой-2005 (ИНН: 7204097730),None,34754,...,nan,72,0,09-12-2022 15:04,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.917243,None,None,Другое
4,"г Ставрополь, ул Шпаковская, Доваторцев, д. 10...",None,None,0,"{'devId': 10296, 'devShortCleanNm': 'CЗ СТРОЙР...",10,2,ООО Стройресурс (ИНН: 2634053119),None,117248,...,"{'objLivElem1KCnt': 16, 'objLivElem2KCnt': 37,...",26,1,02-12-2022 12:57,26-000567,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,0.609216,None,None,Монолит-кирпич
